## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* I/O 操作なし（読み書きは関数外で実施する前提）
* 不要な `print` や `sort_values` は使用しない
* 指定シグネチャはこの回答内で定義するものを前提とする

---

## 1) 問題

### PROBLEM_STATEMENT

Table: `Sales`

```text
+-------------+-------+
| Column Name | Type  |
+-------------+-------+
| sale_id     | int   |
| product_id  | int   |
| year        | int   |
| quantity    | int   |
| price       | int   |
+-------------+-------+
(sale_id, year) が複合主キー
各行は、ある product_id のある year における 1 件の販売を表す
同じ product_id・同じ year で複数行あり得る
```

**要件**

* 各 `product_id` ごとに、「その商品が最初に販売された年（最小の `year`）」を特定する
* その「最初の年」に属する **全ての販売行** を抽出する（集約しない）
* 出力列と順序:

  * `product_id`
  * `first_year`（= その商品の最初の販売年）
  * `quantity`
  * `price`

### 入力 DF

* `sales: pd.DataFrame`

  * 列: `['sale_id', 'product_id', 'year', 'quantity', 'price']` を前提

### 出力

* `pd.DataFrame`

  * 列名と順序は **`['product_id', 'first_year', 'quantity', 'price']`**

---

## 2) 実装（指定シグネチャ厳守）

> 方針はテンプレ通り **列最小化 → グループ処理 → 条件抽出**。
> 「最小年」は `groupby.transform('min')` で各行に持たせ、`year == first_year` の行だけを残します。

```python
import pandas as pd

def find_first_year_sales(sales: pd.DataFrame) -> pd.DataFrame:
    """
    各 product_id について、最初に販売された年に属する全ての販売行を抽出する。

    Args:
        sales (pd.DataFrame): 列 'sale_id', 'product_id', 'year', 'quantity', 'price' を含む入力データ

    Returns:
        pd.DataFrame: 列名と順序は
            ['product_id', 'first_year', 'quantity', 'price']
    """
    # 1) 列最小化（必要な列だけに絞る）
    df = sales[['product_id', 'year', 'quantity', 'price']].copy()

    # 2) グループ処理
    # 各 product_id ごとに最小の year（最初の販売年）を求めて、
    # transform で各行に同じ値を持たせる
    first_year = df.groupby('product_id')['year'].transform('min')

    # 3) 条件抽出：year がその product_id の最初の年と等しい行だけ残す
    kept = df.loc[df['year'].eq(first_year)]

    # 4) 列名整形と最終投影
    out = kept.rename(columns={'year': 'first_year'})[
        ['product_id', 'first_year', 'quantity', 'price']
    ]

    return out

Analyze Complexity
Runtime 316 ms
Beats 59.93%
Memory 69.54 MB
Beats 71.21%

```

* ポイント:

  * `groupby('product_id')['year'].transform('min')` によって、

    * 各行に「その product_id の最小 year」を直接持たせる
  * 同じ `product_id`・同じ `year` の行が複数あっても、

    * その年は最小年であればすべて残る（仕様どおり）

---

## 3) アルゴリズム説明

### 使用 API

* `DataFrame[['col1', ...]]`
  → 列最小化（不要列を落として処理対象を縮小）
* `groupby('product_id')['year'].transform('min')`

  * 各グループの集計値（ここでは最小年）を、グループ内の全行に展開する
  * 戻りは元と同じ長さの `Series`
* `Series.eq(...)`
  → ベクトル化された等価比較（`==` とほぼ同じだが、メソッド形式で読みやすい）
* `DataFrame.loc[mask]`

  * ブールマスクで必要な行のみ抽出
* `DataFrame.rename(columns={...})`

  * 列名の変更 (`year` → `first_year`)
* `DataFrame[...]`（列リスト指定）

  * 最終的な列の順序を仕様どおりに揃える

### NULL / 重複 / 型

* `year` が NULL（`NaN`）の場合

  * `min` は通常、非 NULL 値の最小を返す
  * そのため「year がすべて NaN の product_id」は `first_year` が NaN になり、

    * `year == first_year` も NaN 比較になるため、どの行も選ばれない
  * 実運用で year の NULL が許されないなら、事前に `dropna(subset=['year'])` などで除外可能
* 同一 `product_id`・同一 `year` に複数行ある場合

  * `transform('min')` は同一 product_id の全行に同じ最小年を付与
  * `year == first_year` で、その年の行はすべて残る
  * 問題文どおり「最初の年における全ての販売行」が取得される
* 型

  * `year` は整数列 (`int` / `Int64`) を想定
  * `transform('min')` により返る型は元列に追従するため、特に追加変換は不要

---

## 4) 計算量（概算）

`N` = 行数、`G` = `product_id` のユニーク数とします。

* `df[['product_id', 'year', 'quantity', 'price']]`

  * 列抽出 … **O(N)**（ビュー的だが `copy()` でメモリ確保あり）
* `groupby('product_id')['year'].transform('min')`

  * グループごとの集計はハッシュベース実装想定
  * 時間: おおむね **O(N)**～**O(N log G)** 近辺
  * メモリ: 元と同じ長さの `Series` 分を追加で使用
* `loc[...]` によるフィルタ

  * ブールマスク評価 + 抽出 … **O(N)**
* `rename` と列再配置

  * 軽微（列メタデータの操作が中心）

総じて、支配項は **groupby.transform** で、計算量は概ね **O(N)**〜**O(N log G)**。
`N` が大きくて `G` が相対的に小さい場合でも、実務上は十分スケールする構成です。

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Sales データフレーム]
  B[前処理 列最小化 product_id year quantity price のみ]
  C[グループ処理 product_id ごとに year の最小値を transform]
  D[条件抽出 year が最小年と等しい行だけ残す]
  E[列名整形 year を first_year に変更 列順を揃える]
  F[出力 product_id first_year quantity price]

  A --> B
  B --> C
  C --> D
  D --> E
  E --> F
```

この形であれば、テンプレの「列最小化 → グループ処理 → 条件抽出」の流れを踏襲しつつ、
Pandas 2.2.2 でも素直に動作する実務的な実装になっていると思います。

結論から言うと、そのスコアなら **十分「合格点」かつほぼ最適クラス**で、アルゴリズムを変えて劇的に速くするのは難しいです。
316ms / Beats 59.93%、メモリ 69.54MB / Beats 71.21% は、LeetCode の分布を考えるとかなり健闘しています。

そのうえで「もう少しだけ詰めるとしたら」という観点で、**現行実装の微調整ポイント**を整理します。

---

## 1. 現行実装の良い点の整理

前回お渡しした実装（概略）は：

```python
import pandas as pd

def find_first_year_sales(sales: pd.DataFrame) -> pd.DataFrame:
    df = sales[['product_id', 'year', 'quantity', 'price']].copy()

    first_year = df.groupby('product_id')['year'].transform('min')

    kept = df.loc[df['year'].eq(first_year)]

    out = kept.rename(columns={'year': 'first_year'})[
        ['product_id', 'first_year', 'quantity', 'price']
    ]

    return out
```

これはテンプレの

> 列最小化 → グループ処理（transform） → 条件抽出

を素直にトレースしていて、アルゴリズムとしてはかなりきれいです。

* 余計な列（`sale_id`）は落としている
* `groupby.transform('min')` で「最初の年」を各行に持たせている
* `year == first_year` だけ抽出しているので、同じ年の複数行も漏れなく取得できる

この構造は基本的に変えなくてよいレベルです。

---

## 2. ごく小さい改善案（やるならこの程度）

### 2-1. `.copy()` を外してメモリ削減

`df` に対して「列追加・書き換え」をしていないので、**`copy()` は必須ではありません**。
LeetCode では `SettingWithCopyWarning` も出ませんし、ここはビューのままで構いません。

```python
def find_first_year_sales(sales: pd.DataFrame) -> pd.DataFrame:
    # 1) copy をやめて軽量化
    df = sales[['product_id', 'year', 'quantity', 'price']]

    # 2) グループ処理
    first_year = df.groupby('product_id')['year'].transform('min')

    # 3) 条件抽出
    kept = df.loc[df['year'].eq(first_year)]

    # 4) 列名整形と最終投影
    out = kept.rename(columns={'year': 'first_year'})[
        ['product_id', 'first_year', 'quantity', 'price']
    ]

    return out

Analyze Complexity
Runtime 313 ms
Beats 65.65%
Memory 69.15 MB
Beats 86.03%

```

これで

* 余分なフルコピー分のメモリを節約
* 実行時間も若干ですが改善する可能性あり

とはいえ、**数 % 程度の差に収まることが多い**ので、「絶対やるべき」ほどではありません。

---

### 2-2. `transform('min')` → `groupby.min + merge` の別案

アルゴリズム自体は同じクラスですが、**集約 + 結合パターン**も書き方としてはアリです。

```python
import pandas as pd

def find_first_year_sales(sales: pd.DataFrame) -> pd.DataFrame:
    # 列最小化
    df = sales[['product_id', 'year', 'quantity', 'price']]

    # 各 product_id ごとの最初の年だけを先に集約
    first_years = (
        df.groupby('product_id', as_index=False)['year']
          .min()
          .rename(columns={'year': 'first_year'})
    )

    # product_id と year が first_year に一致する行だけを結合で回収
    merged = df.merge(
        first_years,
        left_on=['product_id', 'year'],
        right_on=['product_id', 'first_year'],
        how='inner'
    )

    out = merged[['product_id', 'first_year', 'quantity', 'price']]

    return out

Analyze Complexity
Runtime 332 ms
Beats 41.75%
Memory 70.03 MB
Beats 37.71%

```

* `groupby.min` → `merge` というオーソドックスな書き方
* データ分布や Pandas の内部実装次第で、

  * `transform('min')` より速くなるケースもあれば
  * 逆に遅くなるケースもあります

LeetCode では「ほぼ誤差レベルの違い」になりがちなので、**可読性の好みで選ぶ**くらいの位置づけです。

---

### 2-3. dtype チューニング（理論上の話）

LeetCode ではあまり期待できませんが、実務では：

* `product_id`, `year`, `quantity`, `price` を `int32` に落とす
* 必要なら `product_id` を `category` にする

などで

* メモリフットプリント削減
* CPU キャッシュ効率向上

が望めます。ただし、LeetCode で毎回 `astype` を噛ますと、そのコストの方が大きくなることも多いので、**ここでは触れない方がむしろ無難**です。

---

## 3. 結論

* 現状のスコア
  → **Runtime 316ms / 59.93%、Memory 69.54MB / 71.21%** は、かなり良い部類
* アルゴリズム的には
  → すでに **O(N)**〜**O(N log G)** の最適クラスで、これ以上大きく改善する余地はほぼない
* やる価値があるとすれば：

  1. `copy()` を外してメモリとわずかな時間を削る
  2. 好みで `groupby.min + merge` パターンを試してみる（速度はケース次第）

ぐらいです。

この問題に関しては、**今の transform 版を「Pandas テンプレ」として採用しておいて十分**だと思います。
他の類題で同じパターン（transform でグループ統計を各行に持たせてフィルタ）を再利用できるので、そこまで仕上がっていればかなり良い状態です。
